In [ ]:
import pandas as pd
from rdkit import Chem
from molop import AutoParser
from rdkit.Chem.Draw import IPythonConsole
from copy import deepcopy
from rxnpredict.geom.ts_gen import read_rct_and_lig,get_amino_type_and_substituent_lst,get_ferr_type_and_substituent_lst,dedup_and_ret_sub_table

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 600, 600

In [206]:
# read TS template
ts_template_Rp_file = "../templated_TS/Template-Rp.gjf"
ts_template_Sp_file = "../templated_TS/Template-Sp.gjf"
ts_template_Rp_file2 = "../templated_TS/Template-Rp2.gjf"
ts_template_Sp_file2 = "../templated_TS/Template-Sp2.gjf"

loc_site_in_template1 = [3,18,20,33,(20,),[11,18],[18,19]]      # 1-index
freeze_site_in_template1 = [[11,18],[18,19],[3,19],[3,33],[3,28],[3,21]]   # 1-index
loc_site_in_template2 = [3,18,20,32,(20,),[11,18],[18,19]]      # 1-index
freeze_site_in_template2 = [[11,18],[18,19],[3,19],[3,32],[3,28],[3,21]]   # 1-index

template_rp = AutoParser(ts_template_Rp_file)[0][0]
template_sp = AutoParser(ts_template_Sp_file)[0][0]
template_rp2 = AutoParser(ts_template_Rp_file2)[0][0]
template_sp2 = AutoParser(ts_template_Sp_file2)[0][0]

MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 135.27it/s]
INFO - 0 files failed to parse, 1 successfully parsed
MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 116.71it/s]
INFO - 0 files failed to parse, 1 successfully parsed
MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 168.53it/s]
INFO - 0 files failed to parse, 1 successfully parsed
MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 147.80it/s]
INFO - 0 files failed to parse, 1 successfully parsed


## Case 1 (Base data)

In [207]:
amino_structure = Chem.MolFromSmarts("O=CNCC(=O)O")
l_amino_substructure = Chem.MolFromSmarts("N[C@@H]([#6])C(=O)O")
d_amino_substructure = Chem.MolFromSmarts("N[C@H]([#6])C(=O)O")
core_fe_1 = Chem.MolFromSmarts("[CX4]-C12C3C4C5C1[Fe]23451678C2C1C6C7C28")
core_fe_2 = Chem.MolFromSmarts("O=C-C12C3C4C5C1[Fe]23451678C2C1C6C7C28")
core_fe = Chem.MolFromSmiles("CCCCN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39")
core_animo = amino_structure

In [260]:
data_file = "../data/sel_base_data_clean.csv"
data_df = pd.read_csv(data_file)

In [209]:
data_df

,Unnamed: 0,Reactant1,Reactant2,Solvents,Solvent1,Solvent2,Reagents,Reagent1,Reagent2,Reagent3,Reagent4,Reagent5,ddG,ee,index,temperature
0,0,O=C(C(=O)c1ccc(-c2ccccc2)cc1)c1ccc(-c2ccccc2)cc1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,C1CCOC1,C1CCOC1,NaN,CC(=O)N[C@@H](Cc1ccccc1)C(=O)O.CC(=O)[O-].CC(=...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],CC(C)(C)OO,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC(=O)N[C@@H](Cc1ccccc1)C(=O)O,2.733668,0.96,0,80
1,1,Cc1ccc(C(=O)C(=O)c2ccc(C)cc2)cc1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,C1CCOC1,C1CCOC1,NaN,CC(=O)N[C@@H](Cc1ccccc1)C(=O)O.CC(=O)[O-].CC(=...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],CC(C)(C)OO,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC(=O)N[C@@H](Cc1ccccc1)C(=O)O,3.227675,0.98,1,80
2,2,O=C(C(=O)c1ccccc1)c1ccccc1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,C1CCOC1,C1CCOC1,NaN,CC(=O)N[C@@H](Cc1ccccc1)C(=O)O.CC(=O)[O-].CC(=...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],CC(C)(C)OO,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC(=O)N[C@@H](Cc1ccccc1)C(=O)O,2.733668,0.96,2,80
3,3,O=C(C(=O)c1ccccc1)c1ccccc1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,C1CCOC1,C1CCOC1,NaN,CC(=O)N[C@H](Cc1ccccc1)C(=O)O.CC(=O)[O-].CC(=O...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],CC(C)(C)OO,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC(=O)N[C@H](Cc1ccccc1)C(=O)O,3.227675,0.98,3,80
4,4,Cc1cccc(C(=O)C(=O)c2cccc(C)c2)c1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,C1CCOC1,C1CCOC1,NaN,CC(=O)N[C@@H](Cc1ccccc1)C(=O)O.CC(=O)[O-].CC(=...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],CC(C)(C)OO,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC(=O)N[C@@H](Cc1ccccc1)C(=O)O,2.939314,0.97,4,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,463,c1ccc2occc2c1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(=O)N(C)C,CC(=O)N(C)C,NaN,CC(=O)[O-].CC(=O)[O-].CC[C@H](C)[C@H](NC(=O)OC...,O=C([O-])[O-].[K+].[K+],O=C1C=CC(=O)C=C1,NaN,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O,3.718090,0.99,829,80
442,464,c1ccc2occc2c1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(=O)N(C)C,CC(=O)N(C)C,NaN,CC(=O)[O-].CC(=O)[O-].CC[C@H](C)[C@H](NC(=O)OC...,O=C([O-])[O-].[K+].[K+],O=C1C=CC(=O)C=C1,NaN,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O,3.718090,0.99,830,80
443,465,c1ccc2occc2c1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7(P(=O)(c1...,CC(=O)N(C)C,CC(=O)N(C)C,NaN,CC(=O)[O-].CC(=O)[O-].CC[C@H](C)[C@H](NC(=O)OC...,O=C([O-])[O-].[K+].[K+],O=C1C=CC(=O)C=C1,NaN,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O,3.718090,0.99,832,80
444,466,c1ccc2occc2c1,CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7([Si](C)(...,CC(=O)N(C)C,CC(=O)N(C)C,NaN,CC(=O)[O-].CC(=O)[O-].CC[C@H](C)[C@H](NC(=O)OC...,O=C([O-])[O-].[K+].[K+],O=C1C=CC(=O)C=C1,NaN,CC(=O)[O-].CC(=O)[O-].[Pd+2],CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O,2.939314,0.97,835,80


In [248]:
rct1_smi_lst,lig_smi_lst = read_rct_and_lig(data_df, rct_col_name="Reactant2",lig_col_name="Reagent5")
# get substituent variation info for different ferrocene and amino
amino_type_lst, amino_substituents_rad_align, max_amino_sub_num = get_amino_type_and_substituent_lst(lig_smi_lst, amino_structure, l_amino_substructure, d_amino_substructure)
fe_type_lst, fe_substituents_rad_align, max_ferr_sub_num = get_ferr_type_and_substituent_lst(rct1_smi_lst, core_fe_1, core_fe_2)
# dedup and get index map
sub_info_df_dep, smi_idx_map = dedup_and_ret_sub_table(rct1_smi_lst,lig_smi_lst,amino_type_lst,fe_type_lst,fe_substituents_rad_align,amino_substituents_rad_align,max_ferr_sub_num,max_amino_sub_num)
idx_smi_map = {value:key for key,value in smi_idx_map.items()}
assert len(fe_type_lst)==len(fe_substituents_rad_align)==len(rct1_smi_lst)

In [250]:
idx_smi_map

{0: 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(C)[C@H](NC(=O)OC(C)(C)C)C(=O)O',
 1: 'CCN(CC)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O',
 2: 'O=C(O)C12C3C4C5C1[Fe]45321678C2C1C6C7C28,O=C(N[C@@H](Cc1ccccc1)C(=O)O)OCc1ccccc1',
 3: 'O=C(O)C12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(C)[C@H](NC(=O)OC(C)(C)C)C(=O)O',
 4: 'O=C(O)C12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC[C@H](C)[C@H](NC(=O)OCc1ccccc1)C(=O)O',
 5: 'CC(N(C)C)C12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(C)C[C@H](NC(=O)OC(C)(C)C)C(=O)O',
 6: 'CC(N(C)C)C12C3C4C5C1[Fe]45321678C2C1C6C7C28,O=C(N[C@@H](Cc1ccccc1)C(=O)O)Oc1ccccc1',
 7: 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,CC(C)(C)OC(=O)N[C@H](C(=O)O)C(C)(C)C',
 8: 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7C28,O=C(N[C@@H](Cc1ccc(C2(C(F)(F)F)N=N2)cc1)C(=O)O)OCC1c2ccccc2-c2ccccc21',
 9: 'CCC12C3C4C5C1[Fe]45321678C2C1C6C7(CN(C)C)C28,CC(C)[C@H](NC(=O)OC(C)(C)C)C(=O)O',
 10: 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7(C(O)(c1ccccc1)c1ccccc1)C28,CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=

In [253]:
## generate initial transition state geometry
for idx,amino_type_,fe_type_,amino_sub_0,amino_sub_1,fe_sub_0,fe_sub_1 in zip(sub_info_df_dep.index.to_list(),sub_info_df_dep["amino_type"].to_list(),sub_info_df_dep["fe_type"].to_list(),
                                                                          sub_info_df_dep["amino_sub_0"].to_list(),sub_info_df_dep["amino_sub_1"].to_list(),
                                                                          sub_info_df_dep["fe_sub_0"].to_list(),sub_info_df_dep["fe_sub_1"].to_list()):
    smi_pair = idx_smi_map[idx]

    fe_sub = [fe_sub_0,fe_sub_1]
    amino_sub = [amino_sub_0,amino_sub_1]
    if amino_type_ == "L":
        #for a_sub in amino_sub:
        if fe_type_ == 1:
            ts_like_rp = (template_rp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=34, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=34, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )

            if smi_pair.split(',')[0] == 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7(CN(C)C)C28':
                freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
                freeze_site_in_template1_cp[3][1] = 44
                loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
                loc_site_in_template1_cp[3] = 44
            else:
                n_atom_idx_rp = sorted([idx for idx in range(len(ts_like_rp.atoms)) if ts_like_rp.atoms[idx] == 7])
                N_cat_idx_rp = n_atom_idx_rp[1]
                n_atom_idx_sp = sorted([idx for idx in range(len(ts_like_sp.atoms)) if ts_like_sp.atoms[idx] == 7])
                N_cat_idx_sp = n_atom_idx_sp[1]

                assert N_cat_idx_rp == N_cat_idx_sp
                freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
                freeze_site_in_template1_cp[3][1] = N_cat_idx_sp + 1
                loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
                loc_site_in_template1_cp[3] = N_cat_idx_sp + 1
            title = write_title(freeze_site_in_template1_cp,loc_site_in_template1_cp)
        elif fe_type_ == 2:
            # for special case
            if fe_sub[0] == '[OH]':
                fe_sub[0] = '[O-]'
            ts_like_rp = (template_rp2
                .replace_substituent("[H]", fe_sub[0], bind_idx=31, angle_split=20)
                .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp2
                .replace_substituent("[H]", fe_sub[0], bind_idx=31, angle_split=20)
                .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            title = write_title(freeze_site_in_template2,loc_site_in_template2)
        
        elif fe_type_ == 3:
            ts_like_rp = (template_rp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=33, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=33, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=29, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            n_atom_idx_rp = sorted([idx for idx in range(len(ts_like_rp.atoms)) if ts_like_rp.atoms[idx] == 7])
            N_cat_idx_rp = n_atom_idx_rp[1]
            n_atom_idx_sp = sorted([idx for idx in range(len(ts_like_sp.atoms)) if ts_like_sp.atoms[idx] == 7])
            N_cat_idx_sp = n_atom_idx_sp[1]

            assert N_cat_idx_rp == N_cat_idx_sp
            freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
            freeze_site_in_template1_cp[3][1] = N_cat_idx_sp + 1
            loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
            loc_site_in_template1_cp[3] = N_cat_idx_sp + 1
            title = write_title(freeze_site_in_template1_cp,loc_site_in_template1_cp)
        else:
            continue
        ts_like_rp.multiplicity = 1
        ts_like_rp.file_path = f"rp_{idx}_{fe_type_}.gjf"
        ts_like_rp.to_GJF_file(
            f"../gen_TS/rp_{idx}_{fe_type_}.gjf",
            template=ts_template_Rp_file
        )
        mod_title_in_gjf(f"../gen_TS/rp_{idx}_{fe_type_}.gjf", title) 
        ts_like_sp.multiplicity = 1
        ts_like_sp.file_path = f"sp_{idx}_{fe_type_}.gjf"
        ts_like_sp.to_GJF_file(
            f"../gen_TS/sp_{idx}_{fe_type_}.gjf",
            template=ts_template_Sp_file
        )
        mod_title_in_gjf(f"../gen_TS/sp_{idx}_{fe_type_}.gjf", title)
        
    elif amino_type_ == "D":
        if fe_type_ == 1:
            ts_like_rp = (template_rp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=34, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=34, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            if smi_pair.split(',')[0] == 'CN(C)CC12C3C4C5C1[Fe]45321678C2C1C6C7(CN(C)C)C28':
                freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
                freeze_site_in_template1_cp[3][1] = 44
                loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
                loc_site_in_template1_cp[3] = 44
            else:
                n_atom_idx_rp = sorted([idx for idx in range(len(ts_like_rp.atoms)) if ts_like_rp.atoms[idx] == 7])
                N_cat_idx_rp = n_atom_idx_rp[1]
                n_atom_idx_sp = sorted([idx for idx in range(len(ts_like_sp.atoms)) if ts_like_sp.atoms[idx] == 7])
                N_cat_idx_sp = n_atom_idx_sp[1]

                assert N_cat_idx_rp == N_cat_idx_sp
                freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
                freeze_site_in_template1_cp[3][1] = N_cat_idx_sp + 1
                loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
                loc_site_in_template1_cp[3] = N_cat_idx_sp + 1
            title = write_title(freeze_site_in_template1_cp,loc_site_in_template1_cp)
        elif fe_type_ == 2:
            # for special case
            if fe_sub[0] == '[OH]':
                fe_sub[0] = '[O-]'
            ts_like_rp = (template_rp2
                .replace_substituent("[H]", fe_sub[0], bind_idx=31, angle_split=20)
                .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp2
                .replace_substituent("[H]", fe_sub[0], bind_idx=31, angle_split=20)
                .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            title = write_title(freeze_site_in_template2,loc_site_in_template2)
        
        elif fe_type_ == 3:
            ts_like_rp = (template_rp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=33, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            ts_like_sp = (template_sp
                        .replace_substituent("[H]", fe_sub[1], bind_idx=33, angle_split=20)
                        .replace_substituent("[H]", fe_sub[0], bind_idx=32, angle_split=20)
                        .replace_substituent("[H]", amino_sub[1], bind_idx=30, angle_split=20)
                        .replace_substituent("[H]", amino_sub[0], bind_idx=28, angle_split=20)
            )
            n_atom_idx_rp = sorted([idx for idx in range(len(ts_like_rp.atoms)) if ts_like_rp.atoms[idx] == 7])
            N_cat_idx_rp = n_atom_idx_rp[1]
            n_atom_idx_sp = sorted([idx for idx in range(len(ts_like_sp.atoms)) if ts_like_sp.atoms[idx] == 7])
            N_cat_idx_sp = n_atom_idx_sp[1]

            assert N_cat_idx_rp == N_cat_idx_sp
            freeze_site_in_template1_cp = deepcopy(freeze_site_in_template1)
            freeze_site_in_template1_cp[3][1] = N_cat_idx_sp + 1
            loc_site_in_template1_cp = deepcopy(loc_site_in_template1)
            loc_site_in_template1_cp[3] = N_cat_idx_sp + 1
            title = write_title(freeze_site_in_template1_cp,loc_site_in_template1_cp)

        else:
            continue
        ts_like_rp.multiplicity = 1
        ts_like_rp.file_path = f"rp_{idx}_{fe_type_}.gjf"
        ts_like_rp.to_GJF_file(
            f"../gen_TS/rp_{idx}_{fe_type_}.gjf",
            template=ts_template_Rp_file
        )
        mod_title_in_gjf(f"../gen_TS/rp_{idx}_{fe_type_}.gjf", title)
        
        ts_like_sp.multiplicity = 1
        ts_like_sp.file_path = f"sp_{idx}_{fe_type_}.gjf"
        ts_like_sp.to_GJF_file(
            f"../gen_TS/sp_{idx}_{fe_type_}.gjf",
            template=ts_template_Sp_file
        )
        mod_title_in_gjf(f"../gen_TS/sp_{idx}_{fe_type_}.gjf", title)

## Case 2 (target data)

In [261]:
data_file = "../data/sel_target_data_clean.csv"
data_df = pd.read_csv(data_file)
data_df

,Reactant1,Reactant2,Solvents,Solvent1,Reagents,Reagent1,Reagent2,Reagent3,Reagent4,Yield,ee,ddG
0,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CCCC[N+](CCCC)(CCCC)CCCC.CC...,CC(=O)O[Pd]OC(C)=O,CCC[C@H](NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],80,86,1.714028
1,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CCCC[N+](CCCC)(CCCC)CCCC.CC...,CC(=O)O[Pd]OC(C)=O,CC[C@H](C)[C@H](NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],73,87,1.766687
2,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)(C)OC(=O)N[C@H](C(=O)O...,CC(=O)O[Pd]OC(C)=O,CC(C)(C)OC(=O)N[C@H](C(=O)O)C(C)(C)C,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],67,80,1.455955
3,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)(C)OC(=O)N[C@@H](CC1CC...,CC(=O)O[Pd]OC(C)=O,CC(C)(C)OC(=O)N[C@@H](CC1CCCCC1)C(=O)O,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],81,88,1.823260
4,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)(C)OC(=O)N[C@@H](CC1CC...,CC(=O)O[Pd]OC(C)=O,CC(C)(C)OC(=O)N[C@@H](CC1CCCC1)C(=O)O,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],78,89,1.884432
5,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)(C)OC(=O)N[C@H](C(=O)O...,CC(=O)O[Pd]OC(C)=O,CC(C)(C)OC(=O)N[C@H](C(=O)O)c1ccccc1,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],73,77,1.352207
6,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)(C)OC(=O)N[C@H](C(=O)O...,CC(=O)O[Pd]OC(C)=O,CC(C)(C)OC(=O)N[C@H](C(=O)O)C1CC1,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],79,88,1.823260
7,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CCCC[N+](CCCC)(CCCC)CCCC.CO...,CC(=O)O[Pd]OC(C)=O,COc1ccc(C[C@H](NC(=O)OC(C)(C)C)C(=O)O)cc1,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],66,87,1.766687
8,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CCCC[N+](CCCC)(CCCC)CCCC.Cc...,CC(=O)O[Pd]OC(C)=O,Cc1ccccc1C[C@H](NC(=O)OC(C)(C)C)C(=O)O,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],77,94,2.303380
9,C=CC(=O)N(C)C,CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39,CN(C)C=O,CN(C)C=O,CC(=O)O[Pd]OC(C)=O.CC(C)[C@H]1CC[C@H](C)C[C@@H...,CC(=O)O[Pd]OC(C)=O,CC(C)[C@H]1CC[C@H](C)C[C@@H]1OC(=O)N[C@@H](Cc1...,O=C([O-])[O-].[K+].[K+],CCCC[N+](CCCC)(CCCC)CCCC.[Br-],73,81,1.493615


In [267]:
amino_structure = Chem.MolFromSmarts("O=CNCC(=O)O")
l_amino_substructure = Chem.MolFromSmarts("N[C@@H]([#6])C(=O)O")
d_amino_substructure = Chem.MolFromSmarts("N[C@H]([#6])C(=O)O")
core_fe = Chem.MolFromSmiles("CN1CC23C4C5C6C2[Fe]56432789C3C2C7C8(C1)C39")
core_animo = amino_structure

In [268]:
ts_template_Rp_file = "../templated_TS/cyclo-TS-Rp_C-H-act-template.gjf"
ts_template_Sp_file = "../templated_TS/cyclo-TS-Sp_C-H_act-template.gjf"
template_rp = AutoParser(ts_template_Rp_file)[0][0]
template_sp = AutoParser(ts_template_Sp_file)[0][0]

loc_site_in_template1 = [30,38,34,11,(34,),[37,38],[1,38]]      # 1-index
freeze_site_in_template1 = [[38,37],[38,1],[30,11],[30,1],[30,32],[30,31]]   # 1-index

MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 89.09it/s]
INFO - 0 files failed to parse, 1 successfully parsed
MolOP parsing with single thread: 100%|██████████| 1/1 [00:00<00:00, 66.20it/s]
INFO - 0 files failed to parse, 1 successfully parsed


In [273]:
fe_substituents_rad_align

[['C[N]C', '[H]'],
 ['CC[N]CC', '[H]'],
 ['[OH]', '[H]'],
 ['[OH]', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', '[CH3]'],
 ['C[N]C', '[CH3]'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[CH2]C'],
 ['C[N]C', 'O[C](c1ccccc1)c1ccccc1'],
 ['[OH]', '[H]'],
 ['C1CC[N]C1', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', '[CH3]'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[CH3]'],
 ['C[N]C', '[H]'],
 ['C[N]C', 'O=[P](c1ccccc1)c1ccccc1'],
 ['C[N]C', '[H]'],
 ['C[N]C', 'C[Si](C)C'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[H]'],
 ['[OH]', '[H]'],
 ['CC[N]CC', '[H]'],
 ['C1CC[N]CC1', '[H]'],
 ['[OH]', '[H]'],
 ['[OH]', '[H]'],
 ['C1CC[N]C1', '[H]'],
 ['CCC[N]C', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', 'C[C](C)O'],
 ['[OH]', '[H]'],
 ['C1COCC[N]1', '[H]'],
 ['C[N]C', '[Br]'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[CH]=O'],
 ['C[N]C', '[H]'],
 ['[OH]', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', '[CH]=C'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[H]'],
 ['[OH]', '[H]'],
 ['C[N]C', '[H]'],
 ['C[N]C', '[H]'],
 ['[OH]', 

In [282]:
rct1_smi_lst,lig_smi_lst = read_rct_and_lig(data_df, rct_col_name="Reactant2",lig_col_name="Reagent2")
# get substituent variation info for different ferrocene and amino
amino_type_lst, amino_substituents_rad_align, max_amino_sub_num = get_amino_type_and_substituent_lst(lig_smi_lst, amino_structure, l_amino_substructure, d_amino_substructure)
fe_type_lst = [1] * len(rct1_smi_lst)
fe_substituents_rad_align = [["[H]"]] * len(rct1_smi_lst)
max_ferr_sub_num = 1
# dedup and get index map
sub_info_df_dep, smi_idx_map = dedup_and_ret_sub_table(rct1_smi_lst,lig_smi_lst,amino_type_lst,fe_type_lst,fe_substituents_rad_align,amino_substituents_rad_align,max_ferr_sub_num,max_amino_sub_num)
idx_smi_map = {value:key for key,value in smi_idx_map.items()}
assert len(fe_type_lst)==len(fe_substituents_rad_align)==len(rct1_smi_lst)

In [283]:
for idx,amino_type,fe_type,amino_sub_0,amino_sub_1 in zip(sub_info_df_dep.index.tolist(),sub_info_df_dep["amino_type"].to_list(),
                                                          sub_info_df_dep["fe_type"].to_list(),sub_info_df_dep["amino_sub_0"].to_list(),
                                                          sub_info_df_dep["amino_sub_1"].to_list()):
    title = write_title(freeze_site_in_template1,loc_site_in_template1)
    amino_sub = [amino_sub_0,amino_sub_1]
    if amino_type == "L":
        #for a_sub in amino_sub:
        ts_like_rp = template_rp.replace_substituent("[H]", amino_sub[0], bind_idx=40, angle_split=20).replace_substituent("[H]", amino_sub[1], bind_idx=39, angle_split=20)
        ts_like_rp.multiplicity = 1
        ts_like_rp.file_path = f"cyc_rp_{idx}.gjf"
        ts_like_rp.to_GJF_file(
            f"../gen_TS/cyc_rp_{idx}.gjf",
            template=ts_template_Rp_file
        )
        mod_title_in_gjf(f"../gen_TS/cyc_rp_{idx}.gjf",title)

        ts_like_sp = template_sp.replace_substituent("[H]", amino_sub[0], bind_idx=40, angle_split=20).replace_substituent("[H]", amino_sub[1], bind_idx=39, angle_split=20)
        ts_like_sp.multiplicity = 1
        ts_like_sp.file_path = f"cyc_sp_{idx}.gjf"
        ts_like_sp.to_GJF_file(
            f"../gen_TS/cyc_sp_{idx}.gjf",
            template=ts_template_Sp_file
        )
        mod_title_in_gjf(f"../gen_TS/cyc_sp_{idx}.gjf",title)

    elif amino_type == "D":
        ts_like_rp = template_rp.replace_substituent("[H]", amino_sub[0], bind_idx=40, angle_split=20).replace_substituent("[H]", amino_sub[1], bind_idx=38, angle_split=20)
        ts_like_rp.multiplicity = 1
        ts_like_rp.file_path = f"cyc_rp_{idx}.gjf"
        ts_like_rp.to_GJF_file(
            f"../gen_TS/cyc_rp_{idx}.gjf",
            template=ts_template_Rp_file
        )
        mod_title_in_gjf(f"../gen_TS/cyc_rp_{idx}.gjf",title)

        ts_like_sp = template_sp.replace_substituent("[H]", amino_sub[0], bind_idx=40, angle_split=20).replace_substituent("[H]", amino_sub[1], bind_idx=38, angle_split=20)
        ts_like_sp.multiplicity = 1
        ts_like_sp.file_path = f"cyc_sp_{idx}.gjf"
        ts_like_sp.to_GJF_file(
            f"../gen_TS/cyc_sp_{idx}.gjf",
            template=ts_template_Sp_file
        )
        mod_title_in_gjf(f"../gen_TS/cyc_sp_{idx}.gjf",title)